# Attentional Factorization Machines: Learning the Weight of Feature Interactions via Attention Networks


https://arxiv.org/pdf/1708.04617

## 研究背景

1. **研究问题**：这篇文章要解决的问题是传统因子机（Factorization Machines, FM）在处理特征交互时，对所有特征交互赋予相同权重，导致模型性能受限。具体来说，FM无法有效区分有用和无用特征之间的交互，可能会引入噪声，降低预测性能。
2. **研究难点**：该问题的研究难点包括：如何有效区分不同特征交互的重要性，如何在建模过程中自动学习特征交互的权重，以及如何在保证模型性能的同时减少模型复杂度。
3. **相关工作**：FM已经被广泛应用于推荐系统、在线广告和图像识别等领域。然而，FM在处理稀疏数据时存在泛化问题。现有的解决方案如多项式回归和宽深模型（Wide&Deep）虽然能够捕捉特征交互，但存在参数估计问题和模型可解释性差的问题。

## 研究方法

**这篇论文提出了一种名为注意力因子机（Attentional Factorization Machine, AFM）的新模型，用于解决FM在处理特征交互时存在的权重分配不均问题。具体来说，AFM通过神经网络中的注意力机制来学习每个特征交互的重要性。**

1. **模型架构**：AFM的神经网络架构包括输入层、嵌入层、成对交互层和基于注意力的池化层。输入层和嵌入层与FM相同，将非零特征嵌入到密集向量中。成对交互层通过元素级乘积扩展特征向量，形成交互向量。基于注意力的池化层通过对交互向量进行加权求和，计算每个特征交互的重要性得分。

![alt text](image/README/1720529217029.png)

2. **成对交互层**：成对交互层的输出表示为：

   $$
   f_{P I}(\mathcal{E})=\left\{\left(v_i\odot v_j\right) x_i x_j\right\}_{(i, j)\in\mathcal{R}_x}
   $$
   其中，$\odot$ 表示两个向量的元素级乘积，$\mathcal{R}_x=\{ (i,j) \}_{i\in \mathcal{X}, j\in \mathcal{X}, j>i}$表示所有可能的特征对。
3. **基于注意力的池化层**：注意力机制通过对交互向量进行加权求和，计算每个特征交互的重要性得分：
   $$
   f_{\text{Att}}(f_{PI}(\mathcal{E}))=\sum_{(i,j)\in \mathcal{R}_x}a_ij(v_i\odot v_j)x_i x_j
   $$

   其中，$a_{ij}$ 是特征交互 $\hat{w}_{i,j}$ 的注意力得分，通过多层感知器（MLP）估计：
   $$
   a'_{ij}=h^T \text{ReLU}(W(v_i\odot v_j)x_i x_j+b)
   $$

   $$
   a_{ij}=\frac{\exp(a'_{ij})}{\sum_{(i,j)\in \mathcal{R}_x}\exp(a'_{ij})}
   $$

   其中，$W\in \mathbb{R}^{t\times k}$, $b\in\mathbb{R}^{t}$, $h\in \mathbb{R}^t$是模型参数，$t$ 表示注意力网络的隐藏层大小。
4. **目标函数与优化**：AFM的目标函数为平方损失：
    $$
    L_r = \sum_{x\in \tau} (\hat{y}_{AFM}(x)-y(x))^2
    $$
    
    其中，$\tau$ 表示训练实例集合。使用随机梯度下降（SGD）优化目标函数，并应用$L_2$正则化和dropout防止过拟合。